In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz

from pprint import pprint
from PIL import Image

import util
import lines
import group
import label
import records

In [2]:
test_files_dir = ".././test_files/"
test_files = util.list_files(test_files_dir)
test_files

['.././test_files/LS_index_1921.pdf',
 '.././test_files/LS_index_1937.pdf',
 '.././test_files/LS_index_1986_split.pdf',
 '.././test_files/Argentinia_LS_index_1934_test_I_DE.pdf',
 '.././test_files/LS_index_1976.pdf',
 '.././test_files/LS_index_1944.pdf',
 '.././test_files/LS_index_1920_split.pdf',
 '.././test_files/LS_index_1986.pdf',
 '.././test_files/LS_index_1920.pdf',
 '.././test_files/LS_index_1944_split.pdf']

In [4]:
pdf_df = util.ocr(test_files[0], 3)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
lines_df

Starting OCR for .././test_files/LS_index_1921.pdf
OCR done for 10 pages.


,line_text,x0,y0,x1,y1,page,dx
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN THE,117,405,1828,446,3,1711
1,ENGLISH EDITION OF THE LEGISLATIVE SERIES OF,117,475,1828,517,3,1711
2,"THE INTERNATIONAL LABOUR OFFICE, rg2r.",316,547,1620,599,3,1304
3,"Note.—T., E., S., or Tl. after the title of a ...",175,642,1827,681,3,1652
4,"text, an extract, a summary, or the title only...",122,690,1828,727,3,1706
...,...,...,...,...,...,...,...
603,URUGUAY,353,2802,533,2833,12,180
604,May 6th Decree issuing regulations in pursuanc...,433,2858,1666,2898,12,1233
605,Act respecting the general compulsory weekly,735,2906,1665,2946,12,930
606,TeSt. Toi cece ee nett ee eee eee nee II Ur. 1,734,2951,1871,2996,12,1137


In [5]:
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)
display(x0_n, bins_x0, bins_x1)

3

,x0,lines,last_x0,count,page
5,"[117, 117, 122, 122, 121, 125, 124, 127, 129, ...","[0, 1, 4, 6, 7, 8, 10, 12, 29, 32]",130,10,3
1,"[209, 212, 215, 215, 215, 215, 215, 216, 216, ...","[13, 16, 21, 27, 30, 33, 37, 40, 42, 44, 47, 4...",217,14,3
0,"[508, 509, 515, 512, 513, 513, 515, 514, 511, ...","[14, 15, 17, 18, 19, 20, 22, 23, 24, 25, 26, 2...",516,27,3
5,"[392, 390, 389, 390, 390, 390, 389, 392, 389]","[56, 65, 68, 69, 72, 74, 90, 110, 112]",389,9,4
4,"[478, 477, 481, 477, 478, 477, 476, 477, 477, ...","[57, 58, 59, 61, 66, 70, 73, 75, 76, 78, 82, 8...",475,24,4
3,"[780, 777, 778, 782, 780, 780, 778, 781, 781, ...","[60, 62, 63, 64, 67, 71, 77, 79, 80, 81, 84, 8...",774,23,4
3,"[138, 141, 142]","[115, 118, 122]",142,3,5
2,"[225, 225, 224, 225, 227, 228, 227, 225, 227, ...","[116, 119, 125, 129, 134, 136, 138, 143, 147, ...",231,17,5
1,"[523, 524, 523, 521, 525, 524, 525, 529, 527, ...","[117, 120, 123, 124, 126, 127, 128, 130, 131, ...",532,40,5
2,"[372, 379, 377, 377]","[183, 194, 198, 226]",377,4,6


,x1,lines,last_x1,count,page
10,"[1440, 1439, 1440, 1443, 1443, 1443, 1443, 144...","[10, 13, 14, 16, 17, 18, 19, 21, 22, 23, 24, 2...",1448,28,3
13,"[1828, 1828, 1827, 1828, 1832, 1830, 1832, 1827]","[0, 1, 3, 4, 5, 6, 7, 9]",1827,8,3
12,"[1620, 1612, 1611, 1612]","[2, 39, 41, 43]",1612,4,3
7,"[1603, 1602, 1602]","[15, 20, 26]",1602,3,3
0,"[1776, 1778]","[52, 55]",1778,2,3
...,...,...,...,...,...
8,1995,575,1995,1,12
11,1838,562,1838,1,12
0,1882,607,1882,1,12
10,1896,565,1896,1,12


In [6]:
page_f = lines_df.loc[lines_df["page"]==3]
x0_b = group.group_lines(page_f, "x0")

In [7]:
x0_b.sort_values(by="last_x0")

,x0,lines,last_x0,count
5,"[117, 117, 122, 122, 121, 125, 124, 127, 129, ...","[0, 1, 4, 6, 7, 8, 10, 12, 29, 32]",130,10
3,"[175, 172, 176]","[3, 5, 9]",176,3
1,"[209, 212, 215, 215, 215, 215, 215, 216, 216, ...","[13, 16, 21, 27, 30, 33, 37, 40, 42, 44, 47, 4...",217,14
4,316,2,316,1
0,"[508, 509, 515, 512, 513, 513, 515, 514, 511, ...","[14, 15, 17, 18, 19, 20, 22, 23, 24, 25, 26, 2...",516,27
2,657,11,657,1


In [8]:
li_df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
l = label.assign_labels(li_df, x0_n)
l.head(20)

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN THE,117,405,1828,446,3,1711,0,1,other
1,ENGLISH EDITION OF THE LEGISLATIVE SERIES OF,117,475,1828,517,3,1711,0,1,other
2,"THE INTERNATIONAL LABOUR OFFICE, rg2r.",316,547,1620,599,3,1304,-1,1,other
3,"Note.—T., E., S., or Tl. after the title of a ...",175,642,1827,681,3,1652,-1,1,other
4,"text, an extract, a summary, or the title only...",122,690,1828,727,3,1706,0,1,other
5,The letters and unbracketed numbers given afte...,172,738,1832,774,3,1660,-1,1,other
6,"the Legislative Series for 1921, which may be ...",122,785,1830,822,3,1708,0,1,other
7,in question or in referring to the Series for ...,121,830,1832,870,3,1711,0,1,other
8,refer to pages in the particular issues of the...,125,879,1126,918,3,1001,0,1,other
9,"The Legislative Series, 1921, having been issu...",176,926,1827,965,3,1651,-1,1,other


In [9]:
# correct x0_type for pages where something went wrong
def correct_x0_types(lines_df, bins_x0, bins_x1, x0_n):
    df = lines_df.copy()
    start_page = lines_df["page"].min()

    text_widths = list(lines.make_borders_df(bins_x0, bins_x1)["dx"]) # difference between mean of first and last bin for x0 for every page

    tw = text_widths.copy()
    tw.sort()
    width_median = tw[int(len(tw)/2)]

    f = filter(lambda w: (not util.similar_to(w, width_median, 5)), text_widths) # filter widths that differ from the rest
    f = list(f)

    p = []
    for w in range(len(f)):
        p.append((f[w], text_widths.index(f[w]) + util.page_start)) # add page to strange width

    p_l = [a + start_page-1 for w, a in p if w<width_median]
    p_g = [a + start_page-1 for w, a in p if w>width_median]
    
    print(f)
    print("Pages small width", p_l)
    print("Pages big width", p_g)

    df.loc[df["page"].isin(p_l) & (df["x0_type"]>=0), "x0_type"] +=1 # correct wrong x0_type for p_l

    # correct wrong x0_type (and x1_type) for p_g
    bins = group.get_line_start_end_bins(df.loc[df["page"].isin(p_g)])
    bins_x0_cor = group.get_relevant_x0_bins(bins[0], x0_n, drop_first=True) # drop bin on the very left
    df_cor = label.assign_types(df.loc[df["page"].isin(p_g)], bins_x0_cor, bins_x1.loc[bins_x1["page"].isin(p_g)], x0_n)
    df.loc[df["page"].isin(p_g), ["x0_type", "x1_type"]] = df_cor[["x0_type", "x1_type"]]

    return df

In [10]:
cli_df = correct_x0_types(li_df, bins_x0, bins_x1, x0_n)
ll_df = label.assign_labels(cli_df, x0_n)

lli_df = ll_df.copy()
lli_df["new_label"] = lli_df["label"]
lli_df = label.improve_country_classification(lli_df)
#ll_df.to_csv("labeling_1986_full.csv")

lli_df.head(50)

[1266.8085106382978, 1323.72, 1324.837837837838]
Pages small width [9]
Pages big width [11, 13]


,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label,new_label
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN THE,117,405,1828,446,3,1711,0,1,other,other
1,ENGLISH EDITION OF THE LEGISLATIVE SERIES OF,117,475,1828,517,3,1711,0,1,other,other
2,"THE INTERNATIONAL LABOUR OFFICE, rg2r.",316,547,1620,599,3,1304,-1,1,other,other
3,"Note.—T., E., S., or Tl. after the title of a ...",175,642,1827,681,3,1652,-1,1,other,other
4,"text, an extract, a summary, or the title only...",122,690,1828,727,3,1706,0,1,other,other
5,The letters and unbracketed numbers given afte...,172,738,1832,774,3,1660,-1,1,other,other
6,"the Legislative Series for 1921, which may be ...",122,785,1830,822,3,1708,0,1,other,other
7,in question or in referring to the Series for ...,121,830,1832,870,3,1711,0,1,other,other
8,refer to pages in the particular issues of the...,125,879,1126,918,3,1001,0,1,other,other
9,"The Legislative Series, 1921, having been issu...",176,926,1827,965,3,1651,-1,1,other,other


In [12]:
l.loc[l["page"]==3]

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
0,I. CHRONOLOGICAL INDEX OF LAWS AND ORDERS IN THE,117,405,1828,446,3,1711,0,1,other
1,ENGLISH EDITION OF THE LEGISLATIVE SERIES OF,117,475,1828,517,3,1711,0,1,other
2,"THE INTERNATIONAL LABOUR OFFICE, rg2r.",316,547,1620,599,3,1304,-1,1,other
3,"Note.—T., E., S., or Tl. after the title of a ...",175,642,1827,681,3,1652,-1,1,other
4,"text, an extract, a summary, or the title only...",122,690,1828,727,3,1706,0,1,other
5,The letters and unbracketed numbers given afte...,172,738,1832,774,3,1660,-1,1,other
6,"the Legislative Series for 1921, which may be ...",122,785,1830,822,3,1708,0,1,other
7,in question or in referring to the Series for ...,121,830,1832,870,3,1711,0,1,other
8,refer to pages in the particular issues of the...,125,879,1126,918,3,1001,0,1,other
9,"The Legislative Series, 1921, having been issu...",176,926,1827,965,3,1651,-1,1,other


In [10]:
lli_df.loc[lli_df["page"]==4]

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label,new_label
56,BELGIUM,392,308,560,336,4,168,0,0,country,country
57,May 24th Act to repeal Section 310 of the Pena...,478,364,2008,404,4,1530,1,1,start,start
58,May 24th Act to guarantec freedom of associati...,477,412,2007,452,4,1530,1,1,start,start
59,June 14th Act to provide for an eight-hour day...,481,455,1716,503,4,1235,1,2,start,start
60,eight hour week. T. ......... 0.22 e eee erenc...,780,502,1873,548,4,1093,2,1,end,end
61,"Aug. 7th Act of 24th December, 1903, respectin...",477,554,1713,592,4,1236,1,2,start,start
62,sation for injurics resulting from industrial,777,600,1715,639,4,938,2,2,middle,middle
63,"accidents, as amended by the Acts of 27th Augu...",778,648,1713,687,4,935,2,2,middle,middle
64,"191g, and 7th August, 921. T............... Be...",782,696,1945,736,4,1163,2,1,end,end
65,BrRaziL,390,746,524,774,4,134,0,0,country,region


In [14]:
rec_gr = records.group_records(lli_df)
rec = records.extract_records(lli_df)
rec.head(40)

,country,region,text,page
0,INTERNATIONAL,,Feb. 9th. Convention respecting insurance agai...,3
1,INTERNATIONAL,,Feb. 14th Convention between France and Belgiu...,3
2,INTERNATIONAL,,Mar. 4/16th Agrecinent respecting the treatmen...,3
3,INTERNATIONAL,,Oct. 8th Convention between Italy and the Unit...,3
4,ARGENTINE,,June 8th Act respecting matches manufactured w...,3
5,AUSTRIA,,Mar. 1oth Act respecting the regulation of emp...,3
6,AUSTRIA,,Mar. 11th Act to amend certain provisions of t...,3
7,AUSTRIA,,Mar. 22nd Act concerning the regulation of agr...,3
8,AUSTRIA,,May 1th Act relating to the contract of servic...,3
9,AUSTRIA,,June rst Act concerning the regulation of empl...,3


In [15]:
rec.to_csv("index_1921.csv")

In [3]:
p_start = 3
pages, dicts = util.read_pdf(test_files[0], p_start)

l_df = lines.make_lines_df(dicts)
l_df = lines.merge_close_lines(l_df)
l_df = lines.remove_useless_lines(l_df)

display(l_df)

Reading pdf from .././test_files/LS_index_1921.pdf
...
Finished reading 10 page(s)


,line_text,spans,x0,y0,x1,y1,page
0,I . CHRONOLOGICA L INDE X O F LAW S AN D ...,"[I, ., CHRONOLOGICA, L, INDE, X, O, F, LAW...",21.12,72.94,331.74,82.94,3
1,ENGLIS H EDITIO N O F TH E LEGISLATIV ...,"[ENGLIS, H, , EDITIO, N, , O, F, , TH, E, ...",21.12,85.42,331.92,95.42,3
2,"TH E INTERNATIONA L LABOU R OFFICE , 1921 .","[TH, E, INTERNATIONA, L, LABOU, R, OFFICE, ...",56.88,98.14,294.78,108.14,3
3,"Note. — T. , E. , S. , o r Tl . afte r ...","[Note., —, T., ,, E., ,, S., ,, o, r, Tl...",31.44,115.99,331.86,122.99,3
4,"text , a n extract , a summary , o r th ...","[text, ,, a, n, extract, ,, a, summary, ,,...",21.84,124.39,331.27,131.39,3
...,...,...,...,...,...,...,...
696,URUGUA Y,"[URUGUA, Y ]",63.12,505.20,96.48,510.80,12
697,Ma y 6t h Decre e issuin g regulation s ...,"[Ma, y, , 6t, h, , Decre, e, issuin, g, re...",78.00,514.71,301.94,521.71,12
698,Ac t respectin g th e genera l compulsor y...,"[Ac, t, respectin, g, th, e, genera, l, co...",132.24,523.11,301.53,530.11,12
699,rest . T I I Ur . 1,"[rest, ., T, , I, I, Ur, ., 1 ]",131.76,532.23,339.06,539.23,12


In [4]:
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(l_df)
borders = lines.make_borders_df(bins_x0, bins_x1)
display(x0_n, bins_x0, bins_x1)

3

,x0,lines,last_x0,count,page
5,"[21.12, 21.12, 21.84, 21.84, 21.84, 22.56, 22....","[0, 1, 4, 6, 7, 8, 10, 11, 16, 18]",23.52,10,3
2,"[37.68, 38.16, 38.64, 38.64, 38.64, 38.64, 38....","[12, 13, 14, 15, 17, 19, 20, 21, 22, 23, 24, 2...",39.12,14,3
0,"[91.68, 91.44, 91.68, 92.16, 92.64, 92.16, 92....","[29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4...",92.88,41,3
3,"[70.56, 70.32, 70.08, 70.08, 70.32, 70.08, 70....","[71, 76, 78, 79, 81, 97, 105, 116, 117]",69.84,9,4
2,"[86.16, 85.92, 86.64, 85.68, 85.92, 85.92, 85....","[72, 73, 74, 75, 77, 80, 82, 98, 99, 100, 101,...",79.92,26,4
1,"[140.4, 140.16, 140.4, 140.4, 139.92, 139.92, ...","[83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 9...",140.16,50,4
2,"[24.96, 25.44, 25.44]","[157, 159, 163]",25.44,3,5
1,"[40.56, 40.56, 40.08, 40.56, 40.8, 41.04, 41.0...","[158, 160, 161, 162, 164, 165, 166, 167, 168, ...",41.52,17,5
0,"[94.56, 94.08, 94.56, 94.32, 93.84, 93.84, 94....","[177, 178, 179, 180, 181, 182, 183, 184, 185, ...",95.52,57,5
4,"[66.72, 67.92, 67.92, 67.68]","[235, 241, 244, 278]",67.68,4,6


,x1,lines,last_x1,count,page
13,"[331.74, 331.92, 331.86, 331.27, 331.53, 331.5...","[0, 1, 3, 4, 5, 6, 7, 9]",330.46,8,3
10,"[365.56, 365.42, 363.22, 365.9, 367.24, 364.66...","[83, 84, 95, 135, 149, 152, 154]",368.5,7,4
1,"[318.1, 317.38, 317.85, 318.25]","[200, 223, 230, 233]",318.25,4,5
3,"[358.66, 360.95, 359.86, 363.21]","[258, 270, 271, 291]",363.21,4,6
2,"[328.66, 328.18, 328.13, 327.94, 327.93, 327.8...","[328, 335, 343, 351, 357, 361, 365, 369]",327.22,8,7
8,"[362.26, 359.61, 359.14, 359.41]","[403, 452, 453, 454]",359.41,4,8
2,"[326.26, 329.14, 332.98, 332.98, 332.74, 332.9...","[465, 486, 490, 494, 498, 502, 505, 509, 512, ...",333.22,11,9
3,"[369.02, 368.78, 368.78, 368.96, 369.66, 369.0...","[524, 528, 531, 543, 547, 550, 554, 559, 562, ...",370.28,12,10
5,"[326.3, 324.82, 326.78, 325.54]","[604, 609, 614, 624]",325.54,4,11
9,"[353.38, 353.14, 356.98, 360.04, 360.04, 359.04]","[648, 653, 664, 678, 683, 689]",359.04,6,12


## Probleme
- Dokumente mit 2 Spalten
- Dokumente mit Länder-Überschrift über ganze Breite der Seite
- Unnötigen Anfangstext auf der ersten Seite rausfiltern
    - 1920

## Fragen
